# 1. Define the Problem

From https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

I like to use a three step process to define the problem. I like to move quickly and I use this mini process to see the problem from a few different perspectives very quickly:

* Step 1: What is the problem? Describe the problem informally and formally and list assumptions and similar problems.
* Step 2: Why does the problem need to be solved? List your motivation for solving the problem, the benefits a solution provides and how the solution will be used.
* Step 3: How would I solve the problem? Describe how the problem would be solved manually to flush domain knowledge.
